In [ ]:
from hydromt_sfincs import SfincsModel
from os.path import join, isfile, basename, isdir
import xarray as xr
import numpy as np
import glob
import hydromt

In [ ]:
events = {'idai': '20190320', 'eloise':'20210125'}
root = r'../../3_models/SFINCS'
mod0 = SfincsModel(join(root, '00_base_100m'), mode='r')

In [ ]:
# read and combine SFINCS outputs
postfix = '_50m'
runs = [path for path in glob.glob(join(root, '*')) if not isfile(path) and basename(path).startswith('1')]
mod0 = SfincsModel(runs[0], mode='r')
print(runs)

for event, date in events.items():
    da_lst, names = [], []
    for run in runs:
        if not isfile(join(run, event, 'sfincs_map.nc')):
            continue
        mod = SfincsModel(join(run, event), mode='r')
        print(run, event)
        mod.read_results()
        hmax = np.maximum(0, mod.results['zsmax'] - mod.results['zb']).sel(timemax=date)
        hmax = hmax.fillna(0).where(mod.results['zb'].notnull(),-9999)
        hmax.raster.set_nodata(-9999)
        # hmax = hmax.raster.reproject_like(mod0.staticmaps, 'min')
        # save geotif
        # hmax.raster.flipud().raster.to_raster(join(mod.root, 'gis', 'hmax.tif'), compress='lzw')
        # append to list
        da_lst.append(hmax.rename('flddph'))
        names.append(basename(run))
    # save combined output to nc
    da = xr.concat(da_lst, dim='run')
    da['run'] = xr.IndexVariable('run', names)
    encoding = {'flddph': {'zlib': True}}
    da.to_netcdf(join(root, f'flddph_{event}{postfix}.nc'))
    # break


In [ ]:
# read and combine CMF outputs
root = r'../../3_models/CMF'

for event, date in events.items():
    da_lst, names = [], []
    for res in ['03min', '06min']:
        runs = [path for path in glob.glob(join(root, res, '*')) if isdir(path)]
        for run in runs:
            fn = join(run, f'flddph_{date}.tif')
            if not isfile(fn): continue
            hmax = hydromt.open_raster(fn).raster.reproject_like(mod0.staticmaps)
            da_lst.append(hmax.rename('flddph'))
            names.append(f'{basename(run)}_{res}')
    # save combined output to nc
    print(names)
    da = xr.concat(da_lst, dim='run')
    da['run'] = xr.IndexVariable('run', names)
    encoding = {'flddph': {'zlib': True}}
    da.to_netcdf(join(root, f'flddph_{event}.nc'))